## Git clone

Cloning this repository after you download this ipynb file.

In [ ]:
import getpass, os

# 这里让你在运行时输入 token, 不会显示在输出里
token = getpass.getpass("input Token: ")
username = "qiulinfan"
owner = "qiulinfan" 
repo = "graphNL2SQL"

#os.environ["GITHUB_TOKEN"] = token
os.environ["GITHUB_USER"] = username
os.environ["GITHUB_OWNER"] = owner
os.environ["GITHUB_REPO"] = repo

!git clone https://github.com/${GITHUB_OWNER}/${GITHUB_REPO}.git
%cd eecs-595-project-nl2sql/

input Token: ··········


In [65]:
!git pull

remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 12 (delta 5), reused 12 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (12/12), 15.99 KiB | 3.20 MiB/s, done.
From https://github.com/qiulinfan/eecs-595-project-nl2sql
   c328ac2..ebb1531  main       -> origin/main
   3b5c631..02f1e57  claude/review-project-docs-01R8zJkwFsnfEdbbDjy63289 -> origin/claude/review-project-docs-01R8zJkwFsnfEdbbDjy63289
Updating c328ac2..ebb1531
Fast-forward
 Prompting.md           | 239 +++++++++++++++++-
 inference.py           | 374 +++++++++++++++++++++++++++++
 requirements_train.txt |  33 +++
 train.py               | 641 +++++++++++++++++++++++++++++++++++++++++++++++++
 train_config.py        | 210 ++++++++++++++++
 5 files changed, 1495 insertions(+), 2 deletions(-)
 create mode 100644 inference.py
 create mode 100644 requirements_train.txt
 create mode 100644 train.py
 create mode 100644 train_

## Download and preprocess data

In [ ]:
!python download_data.py

NL2SQL Dataset Downloader

[1/2] WikiSQL Dataset
----------------------------------------
WikiSQL already downloaded.

[2/2] Spider Dataset
----------------------------------------
Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=759f23ab-dd63-4234-a1b8-8b81f8b23a8c
To: /content/eecs-595-project-nl2sql/data/spider_data.zip
100% 206M/206M [00:01<00:00, 201MB/s]
Extracting Spider dataset...
Renamed spider_data/ -> spider/
Spider extracted to /content/eecs-595-project-nl2sql/data/spider

Dataset Verification
WikiSQL train: 56355 entries
WikiSQL dev: 8421 entries
WikiSQL test: 15878 entries
WikiSQL train_tables: 18585 entries
WikiSQL dev_tables: 2716 entries
WikiSQL test_tables: 5230 entries
Spider train: 7000 entries
Spider dev: 1034 entries
Spider tables: 166 entries

Done!


In [ ]:
!python prepare_training_data.py --semantic  --semantic-threshold 0.8 --wikisql-balanced 5000 --spider

Processing WikiSQL train split...
Sampling 5000 balanced WikiSQL examples from train...
  Pattern distribution in dataset:
    where_only: 40598
    count: 5114
    min: 3231
    max: 3161
    avg: 2201
    sum: 2042
    select_only: 8
  Sampled 4293 examples
WikiSQL: 100% 4293/4293 [00:00<00:00, 39926.71it/s]
Processed 4293 WikiSQL examples
Processing WikiSQL dev split...
Sampling 1000 balanced WikiSQL examples from dev...
  Pattern distribution in dataset:
    where_only: 6017
    count: 779
    max: 507
    min: 468
    avg: 329
    sum: 321
    select_only: 0
  Sampled 999 examples
WikiSQL: 100% 999/999 [00:00<00:00, 44669.25it/s]
Processed 999 WikiSQL examples
Saved 4293 examples to /content/eecs-595-project-nl2sql/training_data/wikisql_train.jsonl
Saved 999 examples to /content/eecs-595-project-nl2sql/training_data/wikisql_dev.jsonl
Processing Spider train split...
Spider:   0% 0/7000 [00:00<?, ?it/s]2025-12-06 12:20:49.661781: E external/local_xla/xla/stream_executor/cuda/cuda_f

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)   # 不省略行
pd.set_option("display.max_columns", None)  # 不省略列
pd.set_option("display.max_colwidth", None) # 不省略内容长度
df = pd.read_json("training_data/spider_train.jsonl", lines=True)
df['schema'][0:10]

,schema
0,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
1,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
2,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
3,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
4,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
5,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
6,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
7,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
8,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
9,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n h

In [ ]:
import pandas as pd
df = pd.read_json("training_data/wikisql_train.jsonl", lines=True)
df['schema'][0:50]

,schema
0,"Schema Graph:\nTable: table(District, s Barangay, Population (2010 census), Area ( has .), Pop. density (per km2))"
1,"Schema Graph:\nTable: table(Rural municipality (RM), RM No., SARM Div. No., Census Div. No., Population (2011), Population (2006), Change (%), Land area (km²), Population density (per km²))"
2,"Schema Graph:\nTable: table(Radical (variants), Stroke count, Pīnyīn, Hiragana - Romaji, Meaning, Frequency, Examples)"
3,"Schema Graph:\nTable: table(South West DFL, Wins, Byes, Losses, Draws, Against)"
4,"Schema Graph:\nTable: table(Pick #, Player, Position, Nationality, NHL team, College/junior/club team)"
5,"Schema Graph:\nTable: table(Player, Attempts, Yards, Average, Long, Touchdowns)"
6,"Schema Graph:\nTable: table(Episode Number, Broadcast Date, Title, Written by, Viewership (Millions))"
7,"Schema Graph:\nTable: table(Ballarat FL, Wins, Byes, Losses, Draws, Against)"
8,"Schema Graph:\nTable: table(Season, Champion, Motorcycle, Wins, 2nd pl., 3rd pl., Team)"
9,"Schema Graph:\nTable: table(HR no., HR name, CR no., LMS no., Built, Works, Withdrawn)"


## Training

### Install Dependencies

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes torch wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.2 MB/s eta 0:00:00


In [ ]:
!pip install -r requirements_train.txt

In [67]:
import transformers
import peft
import accelerate

print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"accelerate: {accelerate.__version__}")

transformers: 4.57.2
peft: 0.18.0
accelerate: 1.12.0


In [78]:
import os
import json
from pathlib import Path
from datetime import datetime

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)

import wandb

### GPU Check

In [74]:
import torch

print("=" * 60)
print("GPU INFORMATION")
print("=" * 60)

if torch.cuda.is_available():
    print(f"✓ CUDA available: True")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    total_mem = props.total_memory / 1e9
    print(f"  Memory: {total_mem:.2f} GB")
    print(f"  CUDA Version: {torch.version.cuda}")

    # Recommend config based on VRAM
    if total_mem < 12:
        print(f"\n  Recommended config: small_gpu (4-bit quantization)")
    elif total_mem < 24:
        print(f"\n Recommended config: default")
    else:
        print(f"\n Recommended config: large_gpu (more LoRA capacity)")
else:
    print("✗ CUDA not available - Training will be VERY slow on CPU")

print("=" * 60)

GPU INFORMATION
✓ CUDA available: True
  GPU: NVIDIA A100-SXM4-80GB
  Memory: 85.17 GB
  CUDA Version: 12.6

 Recommended config: large_gpu (more LoRA capacity)


### Configuration

Define your hyperparameters and LoRA configuration.

In [94]:
# =============================================================================
# MODEL CONFIGURATION
# =============================================================================

# Model selection - choose one:
# - "Qwen/Qwen2.5-7B-Instruct"          (~7B params, recommended)
# - "microsoft/Phi-3.5-mini-instruct"   (~3.8B params, faster)
# - "deepseek-ai/deepseek-coder-6.7b-instruct"  (~6.7B, code-focused)
# - "codellama/CodeLlama-7b-Instruct-hf" (~7B, Meta's code model)
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# Quantization (for memory efficiency)
LOAD_IN_4BIT = True    # Recommended for <16GB VRAM
LOAD_IN_8BIT = False   # Alternative: less compression, slightly better quality

# =============================================================================
# LORA CONFIGURATION
# =============================================================================

LORA_R = 32              # Rank: higher = more capacity, more memory (8, 16, 32, 64)
LORA_ALPHA = 64          # Scaling factor (typically 1-2x r)
LORA_DROPOUT = 0.05      # Dropout for regularization

# Target modules for LoRA
# Full: ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
# Light: ["q_proj", "v_proj"]
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]

# =============================================================================
# TRAINING CONFIGURATION
# =============================================================================

# Training phases
WIKISQL_EPOCHS = 1       # Phase 1: WikiSQL warmup
SPIDER_EPOCHS = 3        # Phase 2: Spider main training

# Batch size
BATCH_SIZE = 2                    # Per-device batch size
GRADIENT_ACCUMULATION = 8         # Effective batch = BATCH_SIZE * GRADIENT_ACCUMULATION

# Learning rate
LEARNING_RATE = 2e-4
LR_SCHEDULER = "cosine"           # "cosine", "linear", "constant"
WARMUP_RATIO = 0.05

# Sequence length
MAX_SEQ_LENGTH = 1024             # Input + Output combined

# Checkpointing
SAVE_STRATEGY = "epoch"           # "epoch" or "steps"
SAVE_TOTAL_LIMIT = 5              # Keep last N checkpoints

# Mixed precision
USE_BF16 = True                   # Use bfloat16 (recommended for newer GPUs)
USE_FP16 = False                  # Use float16 (for older GPUs)

# Gradient checkpointing (saves memory, slightly slower)
GRADIENT_CHECKPOINTING = True

# =============================================================================
# DATA CONFIGURATION
# =============================================================================

DATA_DIR = "./training_data"
OUTPUT_DIR = "./checkpoints"

# Limit samples (None = use all)
MAX_TRAIN_SAMPLES = None          # Set to e.g. 1000 for quick testing
MAX_EVAL_SAMPLES = 500            # Limit eval for speed

# =============================================================================
# WANDB CONFIGURATION
# =============================================================================

USE_WANDB = True
WANDB_PROJECT = "nl2sql-finetuning"
WANDB_RUN_NAME = None             # Auto-generated if None

eval_strategy="epoch",  # ← Now matches
save_strategy="epoch",
load_best_model_at_end=True,


print(" Configuration loaded!")
print(f"  Model: {MODEL_NAME}")
print(f"  LoRA rank: {LORA_R}, alpha: {LORA_ALPHA}")
print(f"  Batch size: {BATCH_SIZE} x {GRADIENT_ACCUMULATION} = {BATCH_SIZE * GRADIENT_ACCUMULATION} effective")
print(f"  Training: {WIKISQL_EPOCHS} epoch WikiSQL + {SPIDER_EPOCHS} epochs Spider")

 Configuration loaded!
  Model: Qwen/Qwen2.5-7B-Instruct
  LoRA rank: 32, alpha: 64
  Batch size: 2 x 8 = 16 effective
  Training: 1 epoch WikiSQL + 3 epochs Spider


### Load Datasets & Run Training

Using encapsulated functions from `training_utils.py`

In [ ]:
from training_utils import (
    TrainingConfig,
    load_datasets,
    show_sample_examples,
    load_model_and_tokenizer,
    setup_lora,
    print_model_parameters,
    init_wandb,
    train_phase1_wikisql,
    train_phase2_spider,
    finish_training,
    get_default_config,
    get_small_gpu_config,
    get_large_gpu_config,
)
print("Training utilities imported!")

In [ ]:
# Choose configuration: get_default_config(), get_small_gpu_config(), get_large_gpu_config()
config = get_default_config()

# Or customize:
# config = TrainingConfig(
#     model_name="Qwen/Qwen2.5-7B-Instruct",
#     lora_r=32,
#     lora_alpha=64,
#     batch_size=2,
#     wikisql_epochs=1,
#     spider_epochs=3,
#     use_wandb=True,
# )

print("Configuration:")
print(f"  Model: {config.model_name}")
print(f"  LoRA rank: {config.lora_r}, alpha: {config.lora_alpha}")
print(f"  Batch size: {config.batch_size} x {config.gradient_accumulation} = {config.batch_size * config.gradient_accumulation} effective")
print(f"  Training: {config.wikisql_epochs} epoch WikiSQL + {config.spider_epochs} epochs Spider")

In [ ]:
# Load datasets
wikisql_train, wikisql_dev, spider_train, spider_dev = load_datasets(config.data_dir)
show_sample_examples(wikisql_train, spider_train)

In [ ]:
# Load model and setup LoRA
model, tokenizer = load_model_and_tokenizer(config)
model = setup_lora(model, config)
print_model_parameters(model)

In [ ]:
# Initialize WandB
run_name = init_wandb(config)

### Phase 1: WikiSQL Warmup

In [ ]:
# Phase 1: WikiSQL Warmup Training
train_phase1_wikisql(model, tokenizer, wikisql_train, wikisql_dev, config, run_name)

### Phase 2: Spider Main Training

In [ ]:
# Phase 2: Spider Main Training
trainer = train_phase2_spider(model, tokenizer, spider_train, spider_dev, config, run_name)

### Finish Training

In [ ]:
# Finish training and cleanup
finish_training(config)

In [ ]:
# Save the final model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

# Finish WandB run
wandb.finish()
print("WandB run finished. Check your dashboard for loss graphs!")

Model saved to ./t5_wikisql_lora_finetuned


train/epoch,▁
train/global_step,▁
total_flos,9297442897920.0
train/epoch,3
train/global_step,3
train_loss,3.20488
train_runtime,3.245
train_samples_per_second,4.622
train_steps_per_second,0.924


WandB run finished. Check your dashboard for loss graphs!


### Download to local

In [ ]:
DOWNLOAD_PATH = "./checkpoints/phase2_spider/final"

def zip_and_download(model_path):
    model_path = Path(model_path)
    if not model_path.exists():
        print(f" Path not found: {model_path}")
        return

    zip_name = f"{model_path.name}_lora_adapter"
    zip_path = f"/tmp/{zip_name}"

    print(f" Creating zip: {model_path}")
    shutil.make_archive(zip_path, 'zip', model_path.parent, model_path.name)

    zip_file = f"{zip_path}.zip"
    print(f"   Size: {os.path.getsize(zip_file)/(1024*1024):.1f} MB")

    try:
        from google.colab import files
        print(" Downloading...")
        files.download(zip_file)
    except ImportError:
        print(f"\n Download manually: {zip_file}")

zip_and_download(DOWNLOAD_PATH)

## Testing

Using encapsulated functions from `testing_utils.py`

In [ ]:
from testing_utils import (
    generate_sql,
    load_finetuned_model,
    evaluate_model,
    show_evaluation_examples,
    run_quick_test,
    run_interactive_test,
    test_with_university_schema,
    test_with_ecommerce_schema,
    UNIVERSITY_SCHEMA,
    ECOMMERCE_SCHEMA,
    SAMPLE_QUESTIONS,
)
print("Testing utilities imported!")

### Load Fine-tuned Model

In [ ]:
# Configuration
CHECKPOINT_PATH = "./checkpoints/phase2_spider/final"

# Load fine-tuned model (or use already loaded model from training)
from pathlib import Path
if Path(CHECKPOINT_PATH).exists():
    eval_model, eval_tokenizer = load_finetuned_model(CHECKPOINT_PATH)
else:
    print(f"Checkpoint not found: {CHECKPOINT_PATH}")
    print("Using model from training session...")
    eval_model, eval_tokenizer = model, tokenizer

### Quick Test

In [ ]:
# Run quick test with default schema/question
run_quick_test(eval_model, eval_tokenizer)

### Evaluate on Spider Dev Set

In [ ]:
# Evaluate on Spider dev set
EVAL_MAX_SAMPLES = 100  # Set to None for full evaluation

if spider_dev:
    eval_results = evaluate_model(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EVAL_MAX_SAMPLES
    )
    
    # Show example predictions
    show_evaluation_examples(eval_results, num_correct=3, num_incorrect=3)
else:
    print("Spider dev data not loaded.")

### Interactive Testing

In [ ]:
# Test with university schema
test_with_university_schema(eval_model, eval_tokenizer)

In [ ]:
# Test with ecommerce schema
test_with_ecommerce_schema(eval_model, eval_tokenizer)

In [ ]:
# Custom test - modify schema and questions as needed
custom_schema = """[TABLES]
employee:
    id (PK)
    name
    department
    salary"""

custom_questions = [
    "What is the average salary?",
    "List employees in the Engineering department.",
    "Who has the highest salary?",
]

run_interactive_test(eval_model, eval_tokenizer, custom_schema, custom_questions)

## Possible Improvement directions



1. **Input Format**: WikiSQL model expects `translate English to SQL: <question> context: <columns>`

2. **More Data**: The sample dataset is tiny. Use hundreds or thousands of examples for real training.

3. **LoRA Rank**: Start with r=16. Increase to 32 or 64 if underfitting, decrease to 8 if overfitting.

4. **Learning Rate**: 1e-4 to 3e-4 typically works well for LoRA.